In [1]:
import pandas as pd
import numpy as np

In [2]:
def fix(path_str):
    return path_str.replace("\\", "/")
def paths():
    x=fix(input())
    print(x)
    return pd.read_csv(x)

In [ ]:
submit = paths()


In [4]:
submit

,id,story,answers
0,XwlpP5otmI,เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต,"[{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]"
1,1pUXZYyXqQ,จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์,[{'จิดาพัฒน์': '[1]'}]
2,drf7jObOL3,กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให...,"[{'กรัณยพร': '[3,7]'}]"
3,ghD7Mhgjsq,คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอม...,NaN
4,5lfeF2x61n,กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้...,NaN
...,...,...,...
995,wEWpIvFoB9,กิมฮัวฉ้อโกงเงินจากลูกค้าผ่านเว็บไซต์ปลอมในงาน...,NaN
996,JbSegK5ioA,กุศลินีบุกรุกเข้าโรงพยาบาล ขโมยยาและทำร้ายพยาบ...,NaN
997,3cnDAIVegR,กลินธ์ขับรถประมาทในการแข่งจนเกิดอุบัติเหตุร้ายแรง,NaN
998,JCTioVnnxg,กชญาณสร้างบริษัทอสังหาริมทรัพย์ปลอม ส่วนจิราทิ...,NaN


# NER

In [5]:
stories = submit['story'].tolist()
stories

['เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต',
 'จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์',
 'กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให้หลายคนบาดเจ็บและเสียชีวิต',
 'คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอมใบเสนอราคา ส่วนกฤดาวัติลักเงินในลิ้นชักบัญชี',
 'กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้ามเกาะกลางชนรถสวนทางจนมีผู้เสียชีวิต 1 ราย บาดเจ็บ 2 ราย',
 'เจษฎาพันธ์ขโมยเงินสดจำนวนมากจากตู้เอทีเอ็มในห้างแล้ววิ่งหนีภายใต้ความมืด',
 'กัณต์พงษ์อ่านข่าวว่าวันนี้มีพายุรุนแรง',
 'จันทน์ศรีขับรถชนกลุ่มคนในงานเทศกาลจนมีผู้เสียชีวิต',
 'จิรัฐิติใช้มีดโจมตีในงานเลี้ยงและขโมยของมีค่าในเวลาเดียวกัน',
 'แก้วเกศขับรถเมาแล้วชนกับคนเดินในตรอก ทำให้เกิดอุบัติเหตุและบาดเจ็บ',
 'คำชวนวางแผนจู่โจมเกตุนภัสหน้าบ้านด้วยอาวุธมีดจนเขาเสียชีวิต',
 'กฤษณ์ชุกรณ์ใช้ปืนข่มขู่และชิงทรัพย์จากพนักงานในร้านกาแฟ ขณะที่งื่อเจ็งบุกรุกเข้าห้องเก็บของและขโมยของมีค่า',
 'กัมมัลยีตปีนขึ้นโครงเหล็กสนามกีฬาชั่วคราวโดยไม่มีอุปกรณ์นิรภัย ทำให้โครงสร้างถล่มใส่งิ้มจูที่อยู่ด้านล่าง',
 'ขจิศักดิ์และเจิดจรัสจัดเวิร์กช็อปเรี

In [ ]:
from openai import OpenAI

prompt = f"""
กรุณาวิเคราะห์ข้อความภาษาไทยต่อไปนี้อย่างละเอียด และระบุชื่อบุคคลทั้งหมด (แค่ชื่อบุคคลเท่านั้น) ด้วย
และใส่ชื่อทุกคนใน list => [people_1, people_2, people_3]
- ตรวจสอบชื่อให้ละเอียด แม้จะเป็นชื่อสั้นๆ เช่น 'แดง', 'ดำ' ก็ต้องใส่ใน list
- อย่าลืมตรวจสอบชื่อที่อาจมีคำนำหน้า เช่น 'นาย', 'นาง', 'นางสาว', 'คุณ', หรือไม่มีคำนำหน้าก็ได้
- หากมีชื่อจริงตามด้วยนามสกุล ให้นับเป็นบุคคลเดียวกัน เช่น 'นายแดง ใจดี'

ไม่ต้องส่งสิ่งอื่นๆมา นอกจาก list ของชื่อคน
ลักษณะของการแยกชื่อบุคคล คือ [ชื่อบุคคล] + กิริยา หรือ กิริยา + [ชื่อบุคคล] เช่น จุฬาภร + ดำเนิน
"""
client = OpenAI(
    api_key="sk-75Gt45VtcvieHJihgyYjFlrc9SHVph5sGMruGCHoffF8bFKc",
    base_url="https://api.opentyphoon.ai/v1"
)

# Make a completion request

text = "เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต"
response = client.chat.completions.create(
    model="typhoon-v2.1-12b-instruct",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"ข้อความ: {text}"}
    ],
    max_tokens=512,
    temperature=0.0
)

# Print the response
resp = response.choices[0].message.content

In [ ]:
import requests

# Prompt (unchanged)
prompt = """
กรุณาวิเคราะห์ข้อความภาษาไทยต่อไปนี้อย่างละเอียด และระบุชื่อบุคคลทั้งหมด (แค่ชื่อบุคคลเท่านั้น) ด้วย
และใส่ชื่อทุกคนใน list => [people_1, people_2, people_3]
- ตรวจสอบชื่อให้ละเอียด แม้จะเป็นชื่อสั้นๆ เช่น 'แดง', 'ดำ' ก็ต้องใส่ใน list
- อย่าลืมตรวจสอบชื่อที่อาจมีคำนำหน้า เช่น 'นาย', 'นาง', 'นางสาว', 'คุณ', หรือไม่มีคำนำหน้าก็ได้
- หากมีชื่อจริงตามด้วยนามสกุล ให้นับเป็นบุคคลเดียวกัน เช่น 'นายแดง ใจดี'

ไม่ต้องส่งสิ่งอื่นๆมา นอกจาก list ของชื่อคน
ลักษณะของการแยกชื่อบุคคล คือ [ชื่อบุคคล] + กิริยา หรือ กิริยา + [ชื่อบุคคล] เช่น จุฬาภร + ดำเนิน
"""

# Text to analyze
text = "เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต"

# Combine prompt and text (just like messages in chat model)
full_prompt = f"{prompt}\nข้อความ: {text}"

# Hugging Face API settings
API_URL = "https://api-inference.huggingface.co/models/deepseek-ai/deepseek-llm-67b-base"
HF_TOKEN = "*"

headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

# Function to call the model
def query(prompt, max_tokens=512, temperature=0.7):
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_tokens,
            "temperature": temperature,
            "return_full_text": False
        }
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Run the query
response = query(full_prompt)

# Extract and print result
print(response[0]['generated_text'])  # May vary based on model structure

In [21]:
resp

'[เคี้ยน, จรา]'

In [29]:
import re
import ast
fixed = re.sub(r'([\wก-๙]+)', r"'\1'", resp)
real_list = ast.literal_eval(fixed)
print(real_list)


['เคี้ยน', 'จรา']


In [ ]:
for story in stories:


# submission

In [33]:
a = [['ner',{"people_1":"เคี้ยน", "people_2":"จรา"}],
 ['ner',{"people_1":"คา", "people_2":"เจรี้ยน"}]]

In [54]:
submit.head(5)

,id,story,answers
0,XwlpP5otmI,เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต,"[{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]"
1,1pUXZYyXqQ,จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์,[{'จิดาพัฒน์': '[1]'}]
2,drf7jObOL3,กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให...,"[{'กรัณยพร': '[3,7]'}]"
3,ghD7Mhgjsq,คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอม...,NaN
4,5lfeF2x61n,กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้...,NaN


In [45]:
stories[:5]

['เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต',
 'จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์',
 'กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให้หลายคนบาดเจ็บและเสียชีวิต',
 'คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอมใบเสนอราคา ส่วนกฤดาวัติลักเงินในลิ้นชักบัญชี',
 'กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้ามเกาะกลางชนรถสวนทางจนมีผู้เสียชีวิต 1 ราย บาดเจ็บ 2 ราย']

In [55]:
print(resp)

```json
[
  {'คำนึงภัช': '[6]'},
  {'กฤดา': '[2,6]'}
]
```


In [ ]:
from os import name
system_prompt = """
คุณคือระบบวิเคราะห์ข้อความทางกฎหมาย ทำหน้าที่อ่านข้อความและจัดประเภทความผิดของแต่ละบุคคล
"""

prompt = f"""
จงวิเคราะห์ข้อความทางกฎหมาย ทำหน้าที่อ่านข้อความและจัดประเภทความผิดของแต่ละบุคคลตามหมวดหมู่ที่กำหนดไว้ด้านล่าง และ ตามบุคคลที่กำหนดไว้ด้านล่างเท่านั้น:
Class 1: ไม่มีความผิดใดๆ เลย (No fault whatsoever)
Class 2: ลักทรัพย์หรือชิงทรัพย์ (Theft or robbery)
Class 3: กระทำโดยประมาท, ละเลย หรือไม่ได้ตั้งใจ ซึ่งทำให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน (Negligence or unintentional harm)
Class 4: บุกรุกเคหสถานหรือบุกรุกเวลากลางคืน (Trespassing on a dwelling or at night)
Class 5: กระทำโดยเจตนา ก่อให้เกิดความเสียหายต่อร่างกาย, จิตใจ หรือทรัพย์สิน (Intentional harm)
Class 6: ฉ้อโกง, ปลอมแปลงเอกสาร หรือสิ่งของเพื่อการค้า (Fraud, forgery, counterfeiting)
Class 7: ทำให้ผู้อื่นเสียชีวิตไม่ว่าในรูปแบบใด (Causing death)

ในข้อความนี้มีบุคคลดังนี้ : {name}
ข้อความที่ต้องแยกแยะ : {story}

โปรดแสดงผลลัพธ์ในรูปแบบ:
[{'ชื่อบุคคล': [เลขคลาสของความผิด]}, ...]
ถ้าบุคคลใดไม่มีความผิด ให้ใส่ [1]
หากบุคคลหนึ่งกระทำผิดหลายข้อ ให้ใส่เลขหลายข้อใน list
ให้ใช้เฉพาะข้อมูลจากข้อความเท่านั้น ห้ามคาดเดาเกินสิ่งที่ระบุ
แยกชื่อบุคคลแต่ละคนให้ชัดเจน และใส่ผลลัพธ์เรียงตามลำดับที่ปรากฏในข้อความ

ตัวอย่าง
ข้อความ: เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต
ผลลัพธ์: [{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]
"""

ans_list = []
for story in stories:
  response = client.chat.completions.create(
    model="typhoon-v2.1-12b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}
    ],
    max_tokens=512,
    temperature=0.0
  )
  resp = response.choices[0].message.content
  ans_list.append(resp)

submit['pred'] = ans_list

In [49]:
submit.head()

,id,story,answers
0,XwlpP5otmI,เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต,"[{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]"
1,1pUXZYyXqQ,จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์,[{'จิดาพัฒน์': '[1]'}]
2,drf7jObOL3,กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให...,"[{'กรัณยพร': '[3,7]'}]"
3,ghD7Mhgjsq,คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอม...,NaN
4,5lfeF2x61n,กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้...,NaN
